In [49]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

sns.set_style("whitegrid")
sns.set_palette("pastel")

In [50]:
df = pd.read_csv("../data/korean_data.tsv", sep="\t")
df.head()

,Rating,Review
0,1,고기가 정말 맛있었어요! 육즙이 가득 있어서 너무 좋았아요 일하시는 분들 너무 친절...
1,1,"잡내 없고 깔끔, 담백한 맛의 순댓국이 순댓국을 안 좋아하는 사람들에게도 술술 넘어..."
2,1,고기 양이 푸짐해서 특 순대국밥을 시킨 기분이 듭니다 맛도 좋습니다 다만 양념장이 ...
3,1,순댓국 자체는 제가 먹어본 순대국밥집 중에서 Top5 안에는 들어요. 그러나 밥 양...
4,1,괜찮아요. 가격 대비 양 많고... 무난한 식당이었네요. 순댓국은 푸짐한 느낌이고 ...


In [51]:
df.shape

(641761, 2)

In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 641761 entries, 0 to 641760
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   Rating  641761 non-null  int64 
 1   Review  641761 non-null  object
dtypes: int64(1), object(1)
memory usage: 9.8+ MB


In [53]:
print(df.duplicated().sum())

1692


In [54]:
### Removing duplicates
df = df.drop_duplicates()
print(df.duplicated().sum())

0


In [57]:
df.shape

(640069, 2)

In [ ]:
# 0 negative, 1 positive 2 neutral/ambiguous
df["Rating"].value_counts()

Rating
1    386865
2    182406
0     70798
Name: count, dtype: int64

In [59]:
df[df["Rating"] == 2].head()

,Rating,Review
8,2,늦은 밤 찾아간 곳. 사람들로 북적북적. 맛있습니다.
9,2,주차된 차도 많고 줄 서서 먹음. 순대는 맛있으나 순댓국에 들어간 고기에서 잡내가 ...
14,2,유명세에 비해 순대 맛은 그저 그런 순대 속이 그냥 당면이다
19,2,"오래되고 유명하다고 해서 가봤는데, 순댓국 맛은 보통이었어요. 직원분들이 많고 친절..."
20,2,"찹쌀순대 좋아해서 가는 곳, 동네에선 예전부터 유명했던 곳이다. 사실 맛이 특별하게..."


In [60]:
## Will not be using neutral reviews
df = df[df["Rating"] != 2]

In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 457663 entries, 0 to 641758
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   Rating  457663 non-null  int64 
 1   Review  457663 non-null  object
dtypes: int64(1), object(1)
memory usage: 10.5+ MB


In [62]:
df["Rating"].value_counts()

Rating
1    386865
0     70798
Name: count, dtype: int64

In [ ]:
## TODO: IMBALANCED